### Business Problem Overview
In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition. So we need to analyse telecom industry data and predict high value customers who are at high risk of churn and identify main indicators of churn. In this project, you will analyse customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn and identify the main indicators of churn.

### Business objective
The business objective is to predict the churn in the last (i.e. the ninth) month using the features/data from the first three months. To do this task well, understanding the typical customer behaviour during churn will be helpful.

### Understanding Customer Behaviour During Churn
Customers usually do not decide to switch to another competitor instantly, but rather over a period of time (this is especially applicable to high-value customers). In churn prediction, we assume that there are three phases of customer lifecycle :

The ‘good’ phase: In this phase, the customer is happy with the service and behaves as usual.

The ‘action’ phase: The customer experience starts to sore in this phase, for e.g. he/she gets a compelling offer from a competitor, faces unjust charges, becomes unhappy with service quality etc. In this phase, the customer usually shows different behaviour than the ‘good’ months. Also, it is crucial to identify high-churn-risk customers in this phase, since some corrective actions can be taken at this point (such as matching the competitor’s offer/improving the service quality etc.)

The ‘churn’ phase: In this phase, the customer is said to have churned. You define churn based on this phase. Also, it is important to note that at the time of prediction (i.e. the action months), this data is not available to you for prediction. Thus, after tagging churn as 1/0 based on this phase, you discard all data corresponding to this phase.

In this case, since you are working over a four-month window, the first two months are the ‘good’ phase, the third month is the ‘action’ phase, while the fourth month is the ‘churn’ phase.

### About dataset:
Dataset contains customer-level information for a span of four consecutive months - June, July, August and September. The months are encoded as 6, 7, 8 and 9, respectively.

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings("ignore")
# Import Libraries
import sys,joblib
import six
sys.modules['sklearn.externals.six'] = six
sys.modules['sklearn.externals.joblib'] = joblib
import numpy as np 
import pandas as pd
import re
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_context("talk", font_scale = 0.65, rc={"grid.linewidth": 5})
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 400)
from sklearn.linear_model import LogisticRegression,LinearRegression,LassoCV,Lasso,Ridge,LogisticRegressionCV
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA,IncrementalPCA
from sklearn.model_selection import GridSearchCV,cross_val_score,KFold,StratifiedKFold,RandomizedSearchCV
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,precision_score,recall_score
from sklearn.metrics import precision_recall_curve,roc_auc_score,roc_curve
from imblearn.over_sampling import SMOTE,RandomOverSampler,ADASYN
from sklearn.preprocessing import StandardScaler,MinMaxScaler,QuantileTransformer
from scipy.stats import skew
from fancyimpute import IterativeImputer,KNN
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import RFE
import statsmodels.api as sm

In [ ]:
telecom = pd.read_csv('./telecom_churn_data.csv')
telecom.head()

In [ ]:
print(telecom.shape)
print('\n')
print(telecom.info(verbose=True, show_counts=True))

In [ ]:
# Summary and checking outliers
telecom.describe()

In [ ]:
# Function to check percentage f null values present in dataset
def calnullpercentage(df):
    missing_num = df[df.columns].isna().sum().sort_values(ascending=False)
    missing_perc = (df[df.columns].isna().sum()/len(df)*100).sort_values(ascending=False)
    missing = pd.concat([missing_num, missing_perc], keys=['Total', 'Percentage'], axis=1)
    missing = missing[missing['Percentage'] > 0]
    # missing['Total'] = missing['Total'].apply(lambda x: f'{x:,}')
    # missing['Percentage'] = missing['Percentage'].apply(lambda x: f'{x:.2f}%')
    return missing

In [ ]:
calnullpercentage(telecom)

In [ ]:
len(calnullpercentage(telecom))

Out of 226 Columns, 166 have null values

In [ ]:
telecom.select_dtypes(include='object').head(3)

##### Filter High-Value Customers
We need to predict churn only for the high-value customers. Define high-value customers as follows: Those who have recharged with an amount more than or equal to X, where X is the 70th percentile of the average recharge amount in the first two months (the good phase).

In [ ]:
# Deriving new columns for total recharge amount data for 6 and 7th month
telecom['tot_rech_amt_data_6'] = telecom['total_rech_data_6'] * telecom['av_rech_amt_data_6']
telecom['tot_rech_amt_data_7'] = telecom['total_rech_data_7'] * telecom['av_rech_amt_data_7']

# Deriving new columns for total amount spent during 6 and 7th month
telecom['tot_amt_6'] = telecom[['total_rech_amt_6', 'tot_rech_amt_data_6']].sum(axis=1)
telecom['tot_amt_7'] = telecom[['total_rech_amt_7', 'tot_rech_amt_data_7']].sum(axis=1)

# First two months average
telecom['avg_amt_6_7'] = telecom[['tot_amt_6', 'tot_amt_7']].mean(axis=1)

# Filtering customers based on percentile havoong goodphase_avg more than or equal to cutoff of 70th percentile
telecom = telecom.loc[(telecom['avg_amt_6_7'] >= np.percentile(telecom['avg_amt_6_7'], 70))]

telecom.shape

I have taken  recharge amountmore than or equal to X, where X is the 70th percentile of the average recharge amount in the first two months and getting 30k rows. If I usemore than(>)sign, will get 29.9k rows, but going with problem statement.

In [ ]:
# Deriving new columns for total recharge amount data for 8 and 9th month
telecom['tot_rech_amt_data_8'] = telecom['total_rech_data_8'] * telecom['av_rech_amt_data_8']
telecom['tot_rech_amt_data_9'] = telecom['total_rech_data_9'] * telecom['av_rech_amt_data_9']

# Deriving new columns for total amount spent during 8 and 9th month
telecom['tot_amt_8'] = telecom[['total_rech_amt_8', 'tot_rech_amt_data_8']].sum(axis=1)
telecom['tot_amt_9'] = telecom[['total_rech_amt_9', 'tot_rech_amt_data_9']].sum(axis=1)

In [ ]:
# Finding categorical columns where dtype is float but those columns are having 0 or 1 values only
cats = []
for col in telecom.columns:
    if len(telecom[col].unique()) == 2 | 3:
        cats.append(col)

# Converting into categorical or object type
telecom[cats] = telecom[cats].apply(lambda x: x.astype('object'))
"""
accessing column using np.r_
`total_rech_num_6` to `total rech_num_9`
`total_rech_data_6` to `total_rech_data_9`
"""
col_tmp = telecom.columns[np.r_[137:141, 161:165]]
telecom[col_tmp] = telecom[col_tmp].apply(lambda x: x.astype('object'))

In [ ]:
x = ['tot_amt_8', 'total_rech_amt_8', 'tot_rech_amt_data_8', 'total_rech_data_8', 'av_rech_amt_data_8']
plt.figure(figsize=(8,5))
fig = sns.heatmap(telecom[x].corr(), annot=True, cmap='flare')
plt.xticks(rotation=45, horizontalalignment='right')
# fig.set_xticklabels(fig.get_xticklabels(), rotation=45, horizontalalignment='right')

Dropping Redundant columns, since we have already created derived features from them and derived features reflects the same information.

In [ ]:
telecom.drop(['tot_rech_amt_data_6', 'tot_rech_amt_data_7','tot_rech_amt_data_8', 'tot_rech_amt_data_9'], inplace=True, axis=1)

### Identifying CHURN CUSTOMERS
Now tag the churned customers (churn=1, else 0) based on the fourth month as follows: Those who have not made any calls (either incoming or outgoing) AND have not used mobile internet even once in the churn phase. The attributes you need to use to tag churners are:

* total_ic_mou_9
* total_og_mou_9
* vol_2g_mb_9
* vol_3g_mb_9

After tagging churners, remove all the attributes corresponding to the churn phase (all attributes having ‘ _9’, etc. in their names).

In [ ]:
# Where summation of columns = 0 then churn = 1 else 0
telecom['churn'] = np.where(telecom[['total_ic_mou_9', 'total_og_mou_9', 'vol_2g_mb_9', 'vol_3g_mb_9']].sum(axis=1) == 0,1,0)

In [ ]:
# Removing all features having '_9', etc. in their names
telecom.drop(telecom.filter(regex='_9|sep', axis=1).columns, axis=1, inplace=True)

In [ ]:
pd.DataFrame(round(telecom['churn'].value_counts(normalize=True)*100,2))

Approximately 92% customers not churned and 8% customers got churned. Also, we can see class imbalance is there and we will deal with it later

For each feature, it counts the values of that feature. If the most recurrent value of the feature is repeated almost in all the instances (**zeros / len(X) * 100 > 95**). Then it drops these features because their values are almost the same for all instances and will not help in learning process and those features are not useful in our prediction.

In [ ]:
telecom.shape

In [ ]:
def redundant_features(df):
    redundant = []
    for i in df.columns:
        counts = df[i].value_counts()
        count_max = counts.iloc[0]
        if count_max / len(df) * 100 > 95:
            redundant.append(i)
    redundant = list(redundant)
    return redundant

In [ ]:
print('Before dropping Redundant features: ', telecom.shape)
redundant_features = redundant_features(telecom)
telecom = telecom.drop(redundant_features, axis=1)
print('After dropping Redundant features: ', telecom.shape)

Function to impute NaN values where %age of missing values > 40%, Reason for taking cutoff 40% is beacuse for these columns we can replace NaN with 0(for example, fb_user_7, not used facebook(NaN),av_rech_amt_data_8, not done recharge(NaN) similarly for other columns.

In [ ]:
""" 
# Function to impute NaN with 0
Function to impute NaN values where %age of missing values > 40%,
Reason for taking cutoff 40% is beacuse for these columns we can replace NaN with 0
(for example, fb_user_7, not used facebook(NaN),av_rech_amt_data_8, not done recharge(NaN) similarly for other columns.
"""

def imputeNaN(df, col_name):
    for col in col_name:
        df[col].fillna(0, inplace=True)

col_40 = calnullpercentage(telecom)[calnullpercentage(telecom)['Percentage'] > 40].index

# Call Function
imputeNaN(telecom, col_40)

In [ ]:
calnullpercentage(telecom)

As we can see from above missing value dataframe and value count == 0, large percentage of values are zero in missing value columns. I can impute most missing value column having NaN value with 0 if I assume that they have not use local incoming service, special outgoing service that is why these columns have NaN values. But this assumption doesn't helping much beacuse most values in these columns have 0 and it infers the same thing. So Imputing missing values for columns mentioned above.

In [ ]:
pd.DataFrame((telecom[calnullpercentage(telecom).index] == 0).sum().head())

In [ ]:
imput_col = list(set(calnullpercentage(telecom).index) - set(('date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8')))
knn_imp = KNNImputer()
telecom[imput_col] = knn_imp.fit_transform(telecom[imput_col])
calnullpercentage(telecom)

In [ ]:
telecom.fillna(0, inplace=True)
# Checking % of null values
calnullpercentage(telecom)

In [ ]:
telecom.shape

In [ ]:
# Checking missing value percentage if any
calnullpercentage(telecom)

In [ ]:
telecom.head()

In [ ]:
# No Duplicate mobile number
len(telecom['mobile_number'].unique())

In [ ]:

telecom.drop(telecom['mobile_number'], inplace=True, axis=1)


In [ ]:
# import pygwalker as pyg
# walker = pyg.walk(telecom, theme_key='vega', dark='light', kernel_computation=True, kanaries_api_key='ak-8ac65ba5e922aa75e5ce395e5042ed3f420d1c8bc6c193882a5c2a24e17105f3')

In [ ]:
telecom.to_csv('C:/There/Stuff/Py/Python/cleanTelecom.csv', index=False)